# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [1]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.array(np.matrix(kernel) ** len(data_set[0]))
    return kernel

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import cvxopt

iris = load_iris()
data_set = iris.data

build_kernel(data_set, kernel_type='poly')

array([[2.72973418e+13, 2.57264295e+13, 2.51470225e+13, ...,
        4.16696015e+13, 4.20374312e+13, 3.91971566e+13],
       [2.57264295e+13, 2.42459217e+13, 2.36998574e+13, ...,
        3.92716193e+13, 3.96182819e+13, 3.69414580e+13],
       [2.51470225e+13, 2.36998574e+13, 2.31660925e+13, ...,
        3.83871263e+13, 3.87259806e+13, 3.61094455e+13],
       ...,
       [4.16696015e+13, 3.92716193e+13, 3.83871263e+13, ...,
        6.36095738e+13, 6.41710882e+13, 5.98353431e+13],
       [4.20374312e+13, 3.96182819e+13, 3.87259806e+13, ...,
        6.41710882e+13, 6.47375597e+13, 6.03635408e+13],
       [3.91971566e+13, 3.69414580e+13, 3.61094455e+13, ...,
        5.98353431e+13, 6.03635408e+13, 5.62850541e+13]])

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [3]:
def choose_set_for_label(data_set, label):
    _data_set = data_set[labels != 2]
    _labels = labels[labels != 2]

    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    _data_set, _labels, test_size=0.2, random_state=15)
    
    train_labels[train_labels < 1] = -1
    test_labels[test_labels < 1] = -1
    
    return train_data_set, test_data_set, train_labels, test_labels

In [4]:
def get_labels_count(data_set):
    return len(data_set)

Use the code that we have implemented earlier:

In [5]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [6]:
iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels)

objects_count = get_labels_count(train_data_set)

lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  9.6305e+01 -1.2289e+03  2e+03  2e-01  2e-15
 1:  5.9143e+01 -1.2031e+02  2e+02  5e-03  2e-15
 2:  7.0898e+00 -1.6497e+01  2e+01  2e-16  2e-15
 3: -5.2057e-01 -3.7668e+00  3e+00  2e-16  7e-16
 4: -1.1712e+00 -1.8374e+00  7e-01  2e-16  3e-16
 5: -1.3952e+00 -1.6846e+00  3e-01  2e-16  2e-16
 6: -1.4671e+00 -1.5679e+00  1e-01  2e-16  2e-16
 7: -1.5060e+00 -1.5164e+00  1e-02  4e-16  3e-16
 8: -1.5105e+00 -1.5106e+00  1e-04  2e-16  3e-16
 9: -1.5105e+00 -1.5105e+00  1e-06  2e-16  2e-16
Optimal solution found.
0.85
